In [3]:
import numpy as np
import gc
import copy
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


from feat_loading import *

In [4]:
app_train = load_dataframe("train_all.csv")
app_test = load_dataframe("test_all.csv")
app_train, app_test = load_extra_feats_post(app_train, app_test)
to_del = del_single_variance_and_nan_too_much(app_train)
app_test = app_test.drop(to_del, axis=1)


BURO_CREDIT_SUM_LIMIT_DEBT_RATIO_VAR
ACTIVE_CREDIT_SUM_LIMIT_DEBT_RATIO_VAR
ACTIVE_MAX_OVERDUE_RATIO_VAR
CLOSED_CREDIT_SUM_LIMIT_DEBT_RATIO_MIN
CLOSED_CREDIT_SUM_LIMIT_DEBT_RATIO_MAX
CLOSED_CREDIT_SUM_LIMIT_DEBT_RATIO_MEAN
CLOSED_CREDIT_SUM_LIMIT_DEBT_RATIO_VAR
CLOSED_CREDIT_SUM_LIMIT_OVERDUE_RATIO_MIN
CLOSED_CREDIT_SUM_LIMIT_OVERDUE_RATIO_MAX
CLOSED_CREDIT_SUM_LIMIT_OVERDUE_RATIO_MEAN
CLOSED_CREDIT_SUM_LIMIT_OVERDUE_RATIO_VAR
PREV_INTREST_RATE_RATIO_MAX
PREV_INTREST_RATE_RATIO_MEAN
PREV_INTREST_RATE_RATIO_VAR
APPROVED_INTREST_RATE_RATIO_MAX
APPROVED_INTREST_RATE_RATIO_MEAN
APPROVED_INTREST_RATE_RATIO_VAR
REFUSED_APP_DURATION_MAX
REFUSED_APP_DURATION_MEAN
REFUSED_APP_PAY_DURATION_MAX
REFUSED_APP_PAY_DURATION_MEAN
REFUSED_INTREST_RATE_RATIO_MAX
REFUSED_INTREST_RATE_RATIO_MEAN
REFUSED_INTREST_RATE_RATIO_VAR
24 columns need to be dropped


In [ ]:
for alg in ["RF", "ET", "CB", "MLP"]:
    oof = load_dataframe(alg + "_oof.csv")
    app_train = app_train.merge(right=oof.reset_index(), how='left', on='SK_ID_CURR')
    del oof
    gc.collect()
    
    t_oof = load_dataframe(alg + "_test.csv")
    app_test = app_test.merge(right=t_oof.reset_index(), how='left', on='SK_ID_CURR')
    del t_oof
    gc.collect()

In [ ]:
import modelling
importlib.reload(modelling)

gc.collect()

cat_feats = ['NAME_CONTRACT_TYPE', 
              'FLAG_OWN_CAR', 
             'FLAG_OWN_REALTY']
submission, fi, metrics = modelling.model(app_train, app_test, cat_feats, parallel=5)
print('Baseline metrics')
print(metrics)
submission.to_csv('stacked.csv', index = False)

c:\program files\python36\lib\site-packages\lightgbm\basic.py:1190: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_CONTRACT_TYPE']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 300 rounds.
[200]	train's binary_logloss: 0.228581	train's auc: 0.81328	valid's binary_logloss: 0.235493	valid's auc: 0.787138
[400]	train's binary_logloss: 0.220008	train's auc: 0.835471	valid's binary_logloss: 0.23508	valid's auc: 0.788239
[600]	train's binary_logloss: 0.212209	train's auc: 0.854873	valid's binary_logloss: 0.234945	valid's auc: 0.788675
[800]	train's binary_logloss: 0.205135	train's auc: 0.87177	valid's binary_logloss: 0.234914	valid's auc: 0.788666
Early stopping, best iteration is:
[645]	train's binary_logloss: 0.210562	train's auc: 0.858917	valid's binary_logloss: 0.234879	valid's auc: 0.788838
Training until validation scores don't improve for 300 rounds.
[200]	train's binary_logloss: 0.228168	train's auc: 0.813255	valid's binary_logloss: 0.23693	valid's auc: 0.786727
[400]	train's binary_logloss: 0.219384	train's auc: 0.836299	valid's binary_logloss: 0.236379	valid's auc: 0.788138
[600]	train's binary_logloss: 0

In [ ]:
importlib.reload(modelling)
fi_sorted = modelling.plot_feature_importances(fi, 100)
# display(fi_sorted[fi_sorted["feature"]=="ORG_XNA_no_Pensioner"])
print(fi_sorted[fi_sorted["importance"]==0]["feature"].tolist())

In [12]:
from sklearn.utils.validation import check_array
